In [1]:
import os
from path import Path
import pickle
import numpy as np
import pandas as pd
import spacy

In [2]:
!python -m spacy download en_core_web_md

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [3]:
data = Path('cleaned.csv')
df = pd.read_csv(data)
df.head()

,type,posts
0,INFJ,The last thing INFJ friend posted his facebo...
1,INFJ,Hello ENFJ EOLdot Sorry hear your distress ...
2,INFJ,Prozac wellbrutin least thirty minutes movin...
3,INFJ,Basically come with three items youve determi...
4,INFJ,All things moderation EOLdot Sims indeed ...


In [4]:
embedding_dim = 300
X = np.empty((df.shape[0], embedding_dim))

In [5]:
def doc2vec(document: str):
    nlp = spacy.load("en_core_web_md")
    doc = nlp(document)
    return doc.vector

In [37]:
for i, doc in enumerate(df["posts"]):
    if i % 1000 == 0:
        print(f"Completed {i} conversions")
    X[i] = doc2vec(doc)

Completed 0 conversions
Completed 1000 conversions
Completed 2000 conversions
Completed 3000 conversions
Completed 4000 conversions
Completed 5000 conversions
Completed 6000 conversions
Completed 7000 conversions
Completed 8000 conversions
Completed 9000 conversions
Completed 10000 conversions
Completed 11000 conversions
Completed 12000 conversions
Completed 13000 conversions
Completed 14000 conversions
Completed 15000 conversions
Completed 16000 conversions
Completed 17000 conversions
Completed 18000 conversions
Completed 19000 conversions
Completed 20000 conversions
Completed 21000 conversions
Completed 22000 conversions
Completed 23000 conversions
Completed 24000 conversions
Completed 25000 conversions
Completed 26000 conversions
Completed 27000 conversions
Completed 28000 conversions
Completed 29000 conversions
Completed 30000 conversions
Completed 31000 conversions
Completed 32000 conversions
Completed 33000 conversions
Completed 34000 conversions
Completed 35000 conversions
Compl

Completed 287000 conversions
Completed 288000 conversions
Completed 289000 conversions
Completed 290000 conversions
Completed 291000 conversions
Completed 292000 conversions
Completed 293000 conversions
Completed 294000 conversions
Completed 295000 conversions
Completed 296000 conversions
Completed 297000 conversions
Completed 298000 conversions
Completed 299000 conversions


In [6]:
import pickle

with open("embeddings.pkl", "wb") as f:
    pickle.dump(X, f)

In [7]:
if os.path.exists("embeddings.pkl"):
    with open("embeddings.pkl", "rb") as f:
        X = pickle.load(f)
else:
    for i, doc in enumerate(df["posts"]):
        if i % 1000 == 0:
            print(f"Completed {i} conversions")
        X[i] = doc2vec(doc)

In [8]:
y = df["type"]

In [9]:
X = X[0 : 10000] 
y = y[0 : 10000]

In [10]:
# Split the dataset into training and testing sets

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y,  random_state=1, stratify=y)
X_train.shape

(7500, 300)

In [11]:
from sklearn.svm import SVC
model = SVC(kernel='linear')

In [12]:
# Train the model
model.fit(X_train, y_train)

SVC(kernel='linear')

In [13]:
# Create predictions with the model
y_pred = model.predict(X_test)
results = pd.DataFrame({
   "Prediction": y_pred,
   "Actual": y_test
}).reset_index(drop=True)
results.head()

,Prediction,Actual
0,INFP,INTP
1,INFP,ENFP
2,INFP,ISTP
3,INFP,INTP
4,INFP,ESTP


In [14]:
# Assess the accuracy_score of the model
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.2288

In [15]:
# Generate confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

        ENFJ       0.00      0.00      0.00        76
        ENFP       0.00      0.00      0.00       158
        ENTJ       0.00      0.00      0.00        91
        ENTP       0.00      0.00      0.00       158
        ESFP       0.00      0.00      0.00         6
        ESTP       0.00      0.00      0.00        13
        INFJ       0.00      0.00      0.00       399
        INFP       0.23      1.00      0.37       572
        INTJ       0.00      0.00      0.00       296
        INTP       0.00      0.00      0.00       486
        ISFJ       0.00      0.00      0.00        40
        ISFP       0.00      0.00      0.00        77
        ISTJ       0.00      0.00      0.00        57
        ISTP       0.00      0.00      0.00        71

    accuracy                           0.23      2500
   macro avg       0.02      0.07      0.03      2500
weighted avg       0.05      0.23      0.09      2500



/Users/josemonagas/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
